
#Upload the File in Google Colab

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# Create the csv file
Not required if the data file is already merged.

In [ ]:
import pandas as pd

# Load the new datasets
files = {
    "RXQ_ANA": "/content/drive/MyDrive/Chronic Pain Management/RXQANA_C.XPT",
    "MPQ": "/content/drive/MyDrive/Chronic Pain Management/MPQ_C.XPT",
    "HSQ": "/content/drive/MyDrive/Chronic Pain Management/HSQ_C.XPT"
}

data = {name: pd.read_sas(file) for name, file in files.items()}

# Display the columns in each dataset
for name, df in data.items():
    print(f"Columns in {name}: {df.columns.tolist()}")

# Update the field mappings based on the actual column names
fields = {
    "RXQ_ANA": ["SEQN", "RXD300", "RXD320", "RXD330", "RXD331Q"],
    "MPQ": ["SEQN", "MPQ110", "MPQ020", "MPQ030"],
    "HSQ": ["SEQN", "HSD010", "HSQ480", "HSQ490"]
}

# Extract the relevant fields from each dataset
extracted_data = {name: df[fields[name]] for name, df in data.items()}

# Merge datasets on the common identifier (SEQN)
merged_data = extracted_data["RXQ_ANA"]
for name, df in extracted_data.items():
    if name != "RXQ_ANA":
        merged_data = pd.merge(merged_data, df, on='SEQN', how='outer')

# Field code to field name mapping
field_names = {
    "RXD300": "Drugs over counter taken regularly",
    "RXD320": "Number of years taking product every day",
    "RXD330": "Currently taking product every day",
    "RXD331Q": "Number of pills/doses taken every day",
    "MPQ110": "Pain duration",
    "MPQ020": "Symptoms present for most of month?",
    "MPQ030": "Symptoms begin only because of injury",
    "HSD010": "General health condition",
    "HSQ480": "Number of days physical health was not good",
    "HSQ490": "Number of days mental health was not good",
}

# Rename the columns in the merged data
merged_data.rename(columns=field_names, inplace=True)


merged_data.head()


Columns in RXQ_ANA: ['SEQN', 'RXD300', 'RXD310', 'RXD320', 'RXD330', 'RXD331Q', 'RXD331U', 'RXQ355', 'RXQ360G', 'RXQ360Q', 'RXQ360U']
Columns in MPQ: ['SEQN', 'MPQ010', 'MPQ020', 'MPQ030', 'MPD040', 'MPD050A', 'MPD050B', 'MPD050C', 'MPD050D', 'MPD050E', 'MPD050F', 'MPD050G', 'MPD050H', 'MPD050I', 'MPD050J', 'MPD050K', 'MPD050L', 'MPD050M', 'MPD050N', 'MPD050O', 'MPD050P', 'MPQ060', 'MPQ070', 'MPQ080', 'MPQ090', 'MPQ100', 'MPQ110', 'MPQ120A', 'MPQ120B', 'MPQ120C', 'MPQ120D', 'MPQ120E', 'MPQ120F', 'MPQ120G', 'MPQ120H', 'MPQ120I', 'MPQ120J', 'MPQ120K', 'MPQ120L', 'MPQ120M', 'MPQ120N', 'MPQ120O', 'MPQ120P', 'MPQ120Q', 'MPQ120R', 'MPQ120S', 'MPQ120T', 'MPQ120U', 'MPQ120V', 'MPQ120W', 'MPQ120X', 'MPQ120Y', 'MPQ120Z', 'MPQ120AA', 'MPQ120AB', 'MPQ120AC', 'MPQ120AD', 'MPQ120AE', 'MPQ120AF']
Columns in HSQ: ['SEQN', 'HSD010', 'HSQ470', 'HSQ480', 'HSQ490', 'HSQ500', 'HSQ510', 'HSQ520', 'HSQ571', 'HSQ580', 'HSQ590', 'HSAQUEX']


,SEQN,Drugs over counter taken regularly,Number of years taking product every day,Currently taking product every day,Number of pills/doses taken every day,Pain duration,Symptoms present for most of month?,Symptoms begin only because of injury,General health condition,Number of days physical health was not good,Number of days mental health was not good
0,21009.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,5.397605e-79
1,21010.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,3.0,1.0,5.397605e-79
2,21012.0,1.0,666.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21015.0,1.0,5.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,2.000000e+00
4,21015.0,1.0,NaN,1.0,2.0,2.0,1.0,2.0,3.0,1.0,2.000000e+00


# Data Preprocessing

## Mapping Values

In [ ]:
# Define a mapping of coded values to their descriptions for each field based on the NHANES documentation
value_descriptions = {
    "Drugs over counter taken regularly": {
        1: "Yes",
        2: "No",
        7: "Refused",
        9: "Don't know"
    },

    "Currently taking product every day": {
        1: "Yes",
        2: "No",
        7: "Refused",
        9: "Don't know"
    },
    "Pain duration": {
        1: "Less than 1 month",
        2: "1 to 3 months",
        3: "More than 3 months",
        4: "Greater than 1 year",
        7: "Refused",
        9: "Don't know"
    },
    "Symptoms present for most of month?": {
        1: "Yes",
        2: "No",
        7: "Refused",
        9: "Don't know",
    },
    "Symptoms begin only because of injury": {
        1: "Yes",
        2: "No",
        7: "Refused",
        9: "Don't know",
    },

    "General health condition": {
        1: "Excellent",
        2: "Very good",
        3: "Good",
        4: "Fair",
        5: "Poor",
        7: "Refused",
        9: "Don't know"
    }
}

# Function to replace coded values with descriptions
def replace_values(df, value_map):
    for column, mapping in value_map.items():
        if column in df.columns:
            df[column] = df[column].map(mapping)
    return df

# Replace coded values with descriptions in the merged dataset
mapped_merged_data = replace_values(merged_data, value_descriptions)

# Save the updated dataset to a CSV file
mapped_merged_data.to_csv('/content/drive/MyDrive/Chronic Pain Management/merged_chronic_pain_data.csv', index=False)

# Display the first few rows of the updated dataset
print(mapped_merged_data.head())

      SEQN Drugs over counter taken regularly  \
0  21009.0                                 No   
1  21010.0                                 No   
2  21012.0                                Yes   
3  21015.0                                Yes   
4  21015.0                                Yes   

   Number of years taking product every day  \
0                                       NaN   
1                                       NaN   
2                                     666.0   
3                                       5.0   
4                                       NaN   

  Currently taking product every day  Number of pills/doses taken every day  \
0                                NaN                                    NaN   
1                                NaN                                    NaN   
2                                 No                                    NaN   
3                                Yes                                    1.0   
4                          

## unique values

In [ ]:
data = mapped_merged_data
# Display unique values for each relevant column
unique_values = {}
for column in data.columns:
    unique_values[column] = data[column].unique()

unique_values

{'SEQN': array([21009., 21010., 21012., ..., 31118., 31122., 31126.]),
 'Drugs over counter taken regularly': array(['No', 'Yes', nan, 'Refused', "Don't know"], dtype=object),
 'Number of years taking product every day': array([ nan, 666.,   5.,   1.,  10.,   8., 999.,   9.,   2.,   4.,   3.,
          7.,  15.,  31.,  20.,   6.,  30.,  12.,  14.,  25.,  11.,  22.,
         13.,  19.,  17.,  23.,  18.,  45.,  40.,  34.,  28.,  16.,  35.,
         50.,  29.,  26.,  21.,  24.]),
 'Currently taking product every day': array([nan, 'No', 'Yes', "Don't know"], dtype=object),
 'Number of pills/doses taken every day': array([nan,  1.,  2.,  3.,  6.,  4.,  8.,  7., 10.,  5.,  9., 23., 20.]),
 'Pain duration': array([nan, 'Less than 1 month', '1 to 3 months', 'Greater than 1 year',
        'More than 3 months', "Don't know"], dtype=object),
 'Symptoms present for most of month?': array([nan, 'Yes', 'No', "Don't know"], dtype=object),
 'Symptoms begin only because of injury': array([nan, 'No', 'Y

## Calculate metrics for monitoring pain levels and tracking treatment outcomes

In [ ]:
import pandas as pd


# Calculate metrics
metrics = {}

# Average Pain Intensity
pain_intensity_counts = data['Pain intensity'].value_counts(normalize=True) * 100
metrics['Average Pain Intensity'] = pain_intensity_counts.to_dict()

# Pain Duration
pain_duration_counts = data['Pain duration'].value_counts(normalize=True) * 100
metrics['Pain Duration'] = pain_duration_counts.to_dict()

# Pain Location
pain_location_counts = data['Pain location'].value_counts(normalize=True) * 100
metrics['Pain Location'] = pain_location_counts.to_dict()

# Pain Interference with Activities
pain_interference_counts = data['Days in past month pain kept from usual activities'].value_counts(normalize=True) * 100
metrics['Pain Interference with Activities'] = pain_interference_counts.to_dict()

# General Health Condition
general_health_counts = data['General health condition'].value_counts(normalize=True) * 100
metrics['General Health Condition'] = general_health_counts.to_dict()

# Use of Pain Relievers
pain_relievers_counts = data['Use of pain relievers in the past month'].value_counts(normalize=True) * 100
metrics['Use of Pain Relievers'] = pain_relievers_counts.to_dict()

# Frequency of Pain Reliever Use
pain_reliever_use_counts = data['Days used pain relievers in the past month'].value_counts(normalize=True) * 100
metrics['Frequency of Pain Reliever Use'] = pain_reliever_use_counts.to_dict()

# Treatment Effectiveness (This requires before and after treatment data, so we will not calculate this metric here)

# Convert metrics dictionary to DataFrame for better visualization
metrics_df = pd.DataFrame(metrics)

# Save metrics to a CSV file
metrics_df.to_csv('/content/drive/MyDrive/Chronic Pain Management _ Tableau/pain_management_metrics.csv', index=True)

# Display the metrics DataFrame
print(metrics_df)

                   Average Pain Intensity  Pain Duration  Pain Location  \
Mild                            70.200573            NaN            NaN   
Very mild                       26.456543            NaN            NaN   
Don't know                       3.342884       0.334288            NaN   
Less than 1 month                     NaN      57.497612            NaN   
1 to 3 months                         NaN      42.168099            NaN   
Head                                  NaN            NaN      30.044843   
Hips                                  NaN            NaN      18.385650   
Neck                                  NaN            NaN      16.591928   
Arms                                  NaN            NaN      15.695067   
Shoulders                             NaN            NaN      11.210762   
Upper back                            NaN            NaN       4.484305   
Hands                                 NaN            NaN       1.793722   
Feet                     